<a href="https://colab.research.google.com/github/rawmean/AppleBadge/blob/master/model_convert_ios_Dec_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 13.4MB/s 
     |████████████████████████████████| 194kB 59.1MB/s 
     |████████████████████████████████| 51kB 9.7MB/s 
     |████████████████████████████████| 1.1MB 58.8MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
Mounted at /content/gdrive


In [2]:
# !pip uninstall coremltools
!pip install -Uqq git+https://github.com/apple/coremltools.git@master

In [3]:
from fastai.vision import *
from fastai.vision.all import *

In [4]:
model_path = "gdrive/MyDrive/Colab/bird_identification/models/bird_model_eport.pkl"
learn = load_learner(model_path, cpu=True)

In [5]:
# - Replace LogSoftmax layer with Softmax at the end to get probability instead of loss/cost
# tmp = list(learn.model.children())[1]
# final_model = list(tmp)[:-1] + [nn.Softmax()]
# final_model = list(learn.model.children()) + [nn.Softmax()]
# final_model = nn.Sequential(*final_model)
# final_model = learn.model

import torch
import torch.nn as nn
import torchvision
final_model = learn.model   # nn.Sequential(learn.model, nn.Softmax())


In [6]:
learn.save('learn_model')
!cp 'models/learn_model.pth' "gdrive/MyDrive/Colab/bird_identification/models/learn_model.pth"

In [39]:
class Flatten(nn.Module):
    "Flatten `x` to a single dimension, often used at the end of a model. `full` for rank-1 tensor"
    def __init__(self, full:bool=False):
        super().__init__()
        self.full = full

    def forward(self, x):
        return x.view(-1) if self.full else x.view(x.size(0), -1)

class AdaptiveConcatPool2d(nn.Module):
    "Layer that concats `AdaptiveAvgPool2d` and `AdaptiveMaxPool2d`." # from pytorch
    def __init__(self, sz:Optional[int]=None): 
        "Output will be 2*sz or 2 if sz is None"
        super().__init__()
        self.output_size = sz or 1
        self.ap = nn.AdaptiveAvgPool2d(self.output_size)
        self.mp = nn.AdaptiveMaxPool2d(self.output_size)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)
    

class BatchNorm1d(torch.nn.BatchNorm2d):
    def forward(self, x):
        x = x.reshape(x.shape[0], -1, 1, 1)
        x = super().forward(x)
        x = x.flatten(1)
        return x


def myhead(nf, nc):
    return \
    nn.Sequential(        # the dropout is needed otherwise you cannot load the weights
            AdaptiveConcatPool2d(),
            Flatten(),
            BatchNorm1d(nf),
            nn.Dropout(p=0.25),
            nn.Linear(nf, 512),
            nn.ReLU(True),
            BatchNorm1d(512),
            nn.Dropout(p=0.5),
            nn.Linear(512, nc),
            nn.Softmax()
        )

In [40]:
nf = 1024; nc = learn.n_out

modules=list(learn.model.children())
modules.pop(-1) 
temp=nn.Sequential(nn.Sequential(*modules))
tempchildren=list(temp.children()) 
tempchildren.append(myhead(nf,nc))
final_model=nn.Sequential(*tempchildren)

In [41]:
final_model[1]

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten()
  (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=1024, out_features=512, bias=True)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=250, bias=True)
  (9): Softmax(dim=None)
)

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten(full=False)
  (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=1024, out_features=512, bias=False)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=250, bias=False)
)

In [42]:
# s0 = list(learn.model.children())[0]
# s1 = list(learn.model.children())[1]
# s1
# s1_aug = list(s1) + [nn.Softmax()]
# s1_aug

# final_model = (nn.Sequential(*s0), nn.Sequential(*s1_aug))
# # final_model = nn.Sequential((*s0), (*s1))

# # final_model = list(learn.model.children()) + [nn.Softmax()]
# final_model = nn.Sequential(*final_model)
# final_model

In [43]:
# Set the model in evaluation mode
final_model.eval();

In [44]:
example_input = torch.randn(1, 3, 224, 224, requires_grad=False).cpu()
traced_model = torch.jit.trace(final_model, example_input)
traced_model.save('traced_model.pth')
!cp 'traced_model.pth' "gdrive/MyDrive/Colab/bird_identification/models/traced_model.pth"

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py:725: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  result = self._slow_forward(*input, **kwargs)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [45]:
# traced_model = torch.jit.load('traced_model.pth')

In [46]:
vocab = learn.dls.vocab
import numpy as np
np.savetxt("class_labels.txt", np.array(vocab), fmt="%s")

In [47]:
vocab

['AFRICAN CROWNED CRANE', 'AFRICAN FIREFINCH', 'ALBATROSS', 'ALEXANDRINE PARAKEET', 'AMERICAN AVOCET', 'AMERICAN BITTERN', 'AMERICAN COOT', 'AMERICAN GOLDFINCH', 'AMERICAN KESTREL', 'AMERICAN PIPIT', 'AMERICAN REDSTART', 'ANHINGA', 'ANNAS HUMMINGBIRD', 'ANTBIRD', 'ARARIPE MANAKIN', 'ASIAN CRESTED IBIS', 'BALD EAGLE', 'BALI STARLING', 'BALTIMORE ORIOLE', 'BANANAQUIT', 'BANDED BROADBILL', 'BAR-TAILED GODWIT', 'BARN OWL', 'BARN SWALLOW', 'BARRED PUFFBIRD', 'BAY-BREASTED WARBLER', 'BEARDED BARBET', 'BELTED KINGFISHER', 'BIRD OF PARADISE', 'BLACK FRANCOLIN', 'BLACK SKIMMER', 'BLACK SWAN', 'BLACK THROATED WARBLER', 'BLACK VULTURE', 'BLACK-CAPPED CHICKADEE', 'BLACK-NECKED GREBE', 'BLACK-THROATED SPARROW', 'BLACKBURNIAM WARBLER', 'BLUE GROUSE', 'BLUE HERON', 'BOBOLINK', 'BROWN NOODY', 'BROWN THRASHER', 'CACTUS WREN', 'CALIFORNIA CONDOR', 'CALIFORNIA GULL', 'CALIFORNIA QUAIL', 'CANARY', 'CAPE MAY WARBLER', 'CAPUCHINBIRD', 'CARMINE BEE-EATER', 'CASPIAN TERN', 'CASSOWARY', 'CHARA DE COLLAR', 'CHI

In [48]:
import coremltools as ct
# Convert to Core ML using the Unified Conversion API

class_labels = [vocab[i] for i in range(learn.n_out)]

red_scale = 1.0 / (0.229 * 255.0)
green_scale = 1.0 / (0.224 * 255.0)
blue_scale = 1.0 / (0.225 * 255.0)

scale = 1.0 / (0.226 * 255.0)

image_input = ct.ImageType(name="input_image", shape=example_input.shape,
                           bias=[-(0.485 * 255.0), -(0.456 * 255.0), -(0.406 * 255.0)],
                           scale=scale,
                           color_layout='RGB')

# args = dict(is_bgr=False, red_bias = -(0.485 * 255.0)  , green_bias = -(0.456 * 255.0)  , blue_bias = -(0.406 * 255.0))
args = dict(is_bgr=False, red_bias = -(0.485 * 255.0)  , green_bias = -(0.456 * 255.0)  , blue_bias = -(0.406 * 255.0), image_scale = scale )
# args = dict(is_bgr=False, image_scale = 1.0/255.0 )

coreml_model = ct.convert(
    traced_model,
    inputs=[image_input], 
    # inputs=[ct.ImageType(name="input.1", shape=example_input.shape)], 
    classifier_config = ct.ClassifierConfig(class_labels),
    # mode='classifier',
    # class_labels="class_labels.txt",
    # preprocessing_args=args, 
)
coreml_model.save("gdrive/MyDrive/Colab/bird_identification/models/newer_bird.mlmodel")


Translating MIL ==> MLModel Ops: 100%|██████████| 492/492 [00:02<00:00, 198.92 ops/s]


In [ ]:
# #spec = coreml_model.get_spec()
# spec = ct.utils.load_spec("gdrive/MyDrive/Colab/bird_identification/models/bird.mlmodel")

# ##check your model input and output
spec = coreml_model.get_spec()
# print(spec.description)
ct.utils.save_spec(spec, "gdrive/MyDrive/Colab/bird_identification/models/new_model.mlmodel")


In [ ]:
print(spec.WhichOneof('Type'))
spec = coreml_model.get_spec()
nn = spec.neuralNetworkClassifier

#look at last few layer types, inputs and output names
for i in range(-1,-4,-1):
  layer = nn.layers[i]
  print(" layer index = %d, Type: %s, input: %s, output: %s" %(i, layer.WhichOneof('layer'), layer.input[0], layer.output[0]))

neuralNetworkClassifier
 layer index = -1, Type: innerProduct, input: input.110, output: 497
 layer index = -2, Type: batchnorm, input: input.109, output: input.110
 layer index = -3, Type: activation, input: input.108, output: input.109


In [ ]:
import copy
spec = coreml_model.get_spec()
nn_spec = spec.neuralNetworkClassifier
layers = nn_spec.layers # this is a list of all the layers
layers_copy = copy.deepcopy(layers) # make a copy of the layers, these will be added back later
del nn_spec.layers[:] # delete all the layers

In [ ]:
# add a scale layer now
# since mlmodel is in protobuf format, we can add proto messages directly
# To look at more examples on how to add other layers: see "builder.py" file in coremltools repo
scale_layer = nn_spec.layers.add()
scale_layer.name = 'scale_layer'
scale_layer.input.append('input_image')
# scale_layer.input.append('input.1')
scale_layer.output.append('input1_scaled')
params = scale_layer.scale
params.scale.floatValue.extend([red_scale, green_scale, blue_scale]) # scale values for RGB
params.shapeScale.extend([3,1,1]) # shape of the scale vector 

# now add back the rest of the layers (which happens to be just one in this case: the crop layer)
nn_spec.layers.extend(layers_copy)

# need to also change the input of the crop layer to match the output of the scale layer
nn_spec.layers[1].input[0] = 'input1_scaled'

In [ ]:
import coremltools.proto.FeatureTypes_pb2 as ft 
##change input and output type to Image
spec = coreml_model.get_spec()
input = spec.description.input[0]
input.type.imageType.colorSpace = ft.ImageFeatureType.RGB
input.type.imageType.height = 224 
input.type.imageType.width = 224

# ct.utils.rename_feature(spec, 'input', 'input_image')

ct.utils.save_spec(spec, "gdrive/MyDrive/Colab/bird_identification/models/UIImageBird_good.mlmodel")

coreml_model = ct.models.MLModel(spec)